In [1]:
import pickle as pkl
import numpy as np
from pathlib import Path
import vggish_input
import params
from tqdm.notebook import tqdm_notebook as tqdm

In [2]:
path_to_final = Path("../../../../Data/FinalDataset/TrainingDataset/")

In [3]:
path_to_save = Path("../../../../Data/FinalDataset/MMExamples")
path_to_save.mkdir(exist_ok=True, parents=True)

In [4]:
sub_srs = [16000, 8000, 4000, 2000, 1000, 500, 250, 125]

In [5]:
# imu params
imu_sr = 50
window_len_imu = 2 * imu_sr  # 2 secs of data worth of samples
hop_len_imu = 10

In [6]:
def frame(data, window_length, hop_length):
    # pad zeros if sequence too short
    if data.shape[0] < window_length:
        len_pad = int(np.ceil(window_length)) - data.shape[0]
        to_pad = np.zeros((len_pad, ) + data.shape[1:])
        data = np.concatenate([data, to_pad], axis=0)
    num_samples = data.shape[0]
    num_frames = 1 + int(np.floor((num_samples - window_length) / hop_length))
    shape = (num_frames, int(window_length)) + data.shape[1:]
    strides = (data.strides[0] * int(hop_length),) + data.strides
    return np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)

In [7]:
for file_path in (path_to_final).iterdir():
    print(f"----{file_path.name}----")
    example_shapes = []
    
    # load data
    with open(file_path, "rb") as f:
        fdata = pkl.load(f)    
        
    # window data
    imu_examples = frame(fdata["IMU"], window_len_imu, hop_len_imu)
    
    og_audio = fdata["Audio"]
    for sub_sr in sub_srs:
        path_to_save_sub_sr = path_to_save / str(sub_sr)
        path_to_save_sub_sr.mkdir(exist_ok=True, parents=True)

        
        # subsample the audio
        factor = 16000 // sub_sr

        wav_data = og_audio[::factor].astype(np.int16)
        
        # create examples from wavfile
        leh = 10
        ueh = sub_sr//2
        audio_examples = vggish_input.wavform_to_concat_examples(wav_data,
                                                                 lower_edge_hertz=leh,
                                                                 upper_edge_hertz=ueh, sr=sub_sr) # shape = samples, num_mel bins
        
        # for each example in imu examples, find the most recent audio example based on timestamp
        audio_timestamps = []
        # get timestamps for each index for audio
        for i in range(audio_examples.shape[0]):
            timestamp = i*params.STFT_HOP_LENGTH_SECONDS + params.STFT_WINDOW_LENGTH_SECONDS
            audio_timestamps.append(timestamp)

        windowed_data_audio = []
        windowed_data_imu = []
        
        for i in range(imu_examples.shape[0]):
            # get end time 
            end_sample_imu = i*hop_len_imu + window_len_imu # units samples
            end_time_imu = end_sample_imu/50
            hop_len_secs_imu = hop_len_imu/50
            window_len_secs_imu = window_len_imu/50

            j = int((i*hop_len_secs_imu + window_len_secs_imu - params.STFT_WINDOW_LENGTH_SECONDS)/params.STFT_HOP_LENGTH_SECONDS)
            
            end_index = j
            example_window_samples = int(params.EXAMPLE_WINDOW_SECONDS/(params.STFT_HOP_LENGTH_SECONDS))
            start_index = end_index - example_window_samples
            if start_index < 0:
                # drop this imu_example as well
                continue
            else:
                audio_example = audio_examples[start_index: end_index]
                # pad
                to_pad = example_window_samples - audio_example.shape[0]
                zero_pad = np.zeros((to_pad, ) + audio_example.shape[1:])
                audio_example = np.concatenate([audio_example, zero_pad], axis=0)
            windowed_data_audio.append(audio_example)
            windowed_data_imu.append(imu_examples[i])
            
        windowed_arr_audio = np.array(windowed_data_audio)
        windowed_arr_imu = np.array(windowed_data_imu)
        
        assert windowed_arr_imu.shape[0] == windowed_arr_audio.shape[0], f"Something's still wrong!, {file.name, audio_examples.shape, imu_examples.shape}"
        example_shapes.append(windowed_arr_audio.shape)
        # print(windowed_arr_audio.shape)
        
        # create dataset
        dataset = {
            # "IMU": imu_examples,
            "IMU": windowed_arr_imu,
            "audio": windowed_arr_audio
        }
        
        # save examples as pkl
        with open(path_to_save_sub_sr / f"{file_path.name[:-4]}.pkl", "wb") as f:
            pkl.dump(dataset, f)
            
    if not(example_shapes.count(example_shapes[0]) == len(example_shapes)):
        print(example_shapes)
    
    break

----49---Kitchen---Chopping---1.pkl----
----201---Kitchen---Grating---3.pkl----
----7---Bathroom---Toilet_flushing---1.pkl----
----7---Kitchen---Pouring_pitcher---2.pkl----
----17---Bathroom---Hair_dryer_in_use---2.pkl----
----205---Bathroom---Washing_hands---3.pkl----
----23---Workshop---Drill in use---3.pkl----
----7---Workshop---Drill in use---2.pkl----
----207---Misc---Laughing---3.pkl----
----53---Bathroom---Toilet_flushing---2.pkl----
----58---Workshop---Sanding---2.pkl----
----203---Kitchen---Chopping---1.pkl----
----103---Kitchen---Pouring_pitcher---2.pkl----
----53---Kitchen---Chopping---2.pkl----
----207---Misc---Scratching---1.pkl----
----23---Bathroom---Toothbrushing---2.pkl----
----41---Misc---Clapping---1.pkl----
----205---Misc---Clapping---3.pkl----
----103---Misc---Alarm_clock---1.pkl----
----207---Kitchen---Pouring_pitcher---1.pkl----
----53---Kitchen---Pouring_pitcher---1.pkl----
----25---Misc---Alarm_clock---3.pkl----
----58---Bathroom---Shaver_in_use---1.pkl----
---

----43---Workshop---Screwing---1.pkl----
----45---Kitchen---Twisting_jar---3.pkl----
----103---Workshop---Sanding---2.pkl----
----53---Kitchen---Blender_in_use---2.pkl----
----55---Kitchen---Washing_Utensils---2.pkl----
----49---Kitchen---Washing_Utensils---2.pkl----
----43---Workshop---Hammering---2.pkl----
----43---Kitchen---Twisting_jar---1.pkl----
----41---Kitchen---Microwave---2.pkl----
----205---Misc---Scratching---2.pkl----
----51---Workshop---Sanding---2.pkl----
----7---Misc---Knocking---1.pkl----
----49---Kitchen---Microwave---2.pkl----
----58---Misc---Drinking---2.pkl----
----47---Bathroom---Brushing_hair---1.pkl----
----49---Misc---Drinking---2.pkl----
----43---Workshop---Drill in use---2.pkl----
----7---Bathroom---Brushing_hair---2.pkl----
----41---Kitchen---Grating---3.pkl----
----23---Kitchen---Microwave---1.pkl----
----47---Other---Other---34.pkl----
----201---Kitchen---Washing_Utensils---1.pkl----
----105---Kitchen---Microwave---3.pkl----
----47---Workshop---Drill in us

----7---Kitchen---Microwave---2.pkl----
----49---Workshop---Drill in use---3.pkl----
----7---Kitchen---Blender_in_use---1.pkl----
----105---Kitchen---Chopping---3.pkl----
----105---Other---Other---34.pkl----
----23---Kitchen---Microwave---3.pkl----
----23---Kitchen---Wiping_with_rag---2.pkl----
----58---Workshop---Vacuum in use---2.pkl----
----201---All---Other---3.pkl----
----7---Kitchen---Blender_in_use---2.pkl----
----51---Workshop---Hammering---3.pkl----
----51---Workshop---Screwing---1.pkl----
----105---Bathroom---Hair_dryer_in_use---1.pkl----
----207---All---Other---3.pkl----
----43---Bathroom---Washing_hands---1.pkl----
----25---Kitchen---Washing_Utensils---2.pkl----
----207---Misc---Clapping---2.pkl----
----25---Bathroom---Toothbrushing---2.pkl----
----45---Kitchen---Wiping_with_rag---3.pkl----
----51---Bathroom---Brushing_hair---2.pkl----
----207---Bathroom---Shaver_in_use---1.pkl----
----47---Misc---Drinking---1.pkl----
----103---Kitchen---Wiping_with_rag---1.pkl----
----51--

----23---Kitchen---Blender_in_use---3.pkl----
----207---Other---Other---58.pkl----
----45---Kitchen---Microwave---3.pkl----
----47---Misc---Alarm_clock---1.pkl----
----207---Bathroom---Toothbrushing---2.pkl----
----49---Bathroom---Hair_dryer_in_use---2.pkl----
----105---Bathroom---Hair_dryer_in_use---3.pkl----
----37---Bathroom---Hair_dryer_in_use---1.pkl----
----105---Bathroom---Toilet_flushing---1.pkl----
----51---Kitchen---Chopping---2.pkl----
----41---Kitchen---Grating---1.pkl----
----207---Misc---Coughing---2.pkl----
----37---Misc---Coughing---3.pkl----
----37---Kitchen---Microwave---1.pkl----
----105---Kitchen---Blender_in_use---2.pkl----
----55---Misc---Laughing---1.pkl----
----7---Bathroom---Washing_hands---1.pkl----
----41---Misc---Scratching---3.pkl----
----203---Kitchen---Pouring_pitcher---3.pkl----
----25---Misc---Drinking---3.pkl----
----47---Workshop---Sanding---3.pkl----
----7---All---Other---1.pkl----
----58---Misc---Scratching---2.pkl----
----203---Kitchen---Microwave-

----53---Workshop---Screwing---2.pkl----
----207---Misc---Alarm_clock---3.pkl----
----49---Other---Other---12.pkl----
----45---Kitchen---Blender_in_use---1.pkl----
----23---Misc---Drinking---2.pkl----
----43---Kitchen---Blender_in_use---2.pkl----
----58---Bathroom---Hair_dryer_in_use---2.pkl----
----25---Kitchen---Twisting_jar---2.pkl----
----25---Kitchen---Wiping_with_rag---1.pkl----
----45---Bathroom---Toilet_flushing---2.pkl----
----7---Misc---Drinking---3.pkl----
----47---Misc---Scratching---2.pkl----
----25---Misc---Drinking---1.pkl----
----25---All---Other---1.pkl----
----41---Workshop---Drill in use---1.pkl----
----207---Bathroom---Brushing_hair---3.pkl----
----55---Kitchen---Chopping---3.pkl----
----37---Kitchen---Microwave---3.pkl----
----58---Bathroom---Washing_hands---2.pkl----
----23---Misc---Laughing---2.pkl----
----17---Kitchen---Pouring_pitcher---3.pkl----
----203---Misc---Knocking---3.pkl----
----7---Misc---Coughing---2.pkl----
----203---Workshop---Hammering---1.pkl----

----23---Workshop---Screwing---2.pkl----
----41---Workshop---Vacuum in use---1.pkl----
----53---Kitchen---Washing_Utensils---1.pkl----
----55---Misc---Clapping---2.pkl----
----103---Kitchen---Microwave---2.pkl----
----45---Kitchen---Pouring_pitcher---3.pkl----
----47---Kitchen---Blender_in_use---1.pkl----
----17---Bathroom---Shaver_in_use---1.pkl----
----51---Workshop---Vacuum in use---1.pkl----
----41---Bathroom---Washing_hands---2.pkl----
----17---Kitchen---Chopping---3.pkl----
----51---Kitchen---Blender_in_use---2.pkl----
----58---Other---Other---49.pkl----
----41---Kitchen---Pouring_pitcher---2.pkl----
----203---Kitchen---Washing_Utensils---1.pkl----
----51---Kitchen---Wiping_with_rag---2.pkl----
----43---Workshop---Drill in use---1.pkl----
----37---Bathroom---Washing_hands---1.pkl----
----37---Workshop---Sanding---1.pkl----
----51---Kitchen---Microwave---3.pkl----
----49---Misc---Knocking---3.pkl----
----201---Kitchen---Grating---2.pkl----
----23---Misc---Scratching---2.pkl----
--

----23---Workshop---Hammering---1.pkl----
----103---Kitchen---Blender_in_use---1.pkl----
----201---Other---Other---12.pkl----
----23---Misc---Knocking---2.pkl----
----201---Misc---Coughing---2.pkl----
----41---Misc---Scratching---1.pkl----
----51---Kitchen---Grating---2.pkl----
----23---Other---Other---59.pkl----
----53---Kitchen---Pouring_pitcher---3.pkl----
----47---Kitchen---Grating---3.pkl----
----205---Misc---Scratching---3.pkl----
----105---Kitchen---Blender_in_use---3.pkl----
----47---Misc---Scratching---1.pkl----
----58---Bathroom---Toilet_flushing---3.pkl----
----7---Bathroom---Hair_dryer_in_use---1.pkl----
----53---Misc---Alarm_clock---1.pkl----
----103---Kitchen---Twisting_jar---2.pkl----
----51---Bathroom---Toothbrushing---1.pkl----
----37---Bathroom---Shaver_in_use---2.pkl----
----7---Bathroom---Hair_dryer_in_use---3.pkl----
----205---Kitchen---Blender_in_use---1.pkl----
----51---Workshop---Vacuum in use---3.pkl----
----45---Kitchen---Microwave---2.pkl----
----205---Bathro

----43---Misc---Alarm_clock---2.pkl----
----203---Kitchen---Twisting_jar---1.pkl----
----23---Misc---Laughing---1.pkl----
----43---Workshop---Sanding---3.pkl----
----43---Bathroom---Hair_dryer_in_use---1.pkl----
----47---Workshop---Drill in use---1.pkl----
----55---Misc---Knocking---2.pkl----
----201---Workshop---Sanding---3.pkl----
----51---Kitchen---Microwave---2.pkl----
----55---Bathroom---Shaver_in_use---1.pkl----
----58---Kitchen---Microwave---3.pkl----
----103---Bathroom---Toothbrushing---1.pkl----
----103---Misc---Clapping---2.pkl----
----45---Kitchen---Washing_Utensils---1.pkl----
----41---Kitchen---Grating---2.pkl----
----49---Kitchen---Twisting_jar---3.pkl----
----103---Other---Other---12.pkl----
----25---Kitchen---Blender_in_use---2.pkl----
----17---Other---Other---19.pkl----
----201---Kitchen---Grating---1.pkl----
----51---Kitchen---Washing_Utensils---1.pkl----
----17---Bathroom---Toothbrushing---2.pkl----
----103---Bathroom---Toilet_flushing---1.pkl----
----105---Kitchen--

----37---Misc---Coughing---1.pkl----
----43---Bathroom---Toothbrushing---1.pkl----
----51---Bathroom---Toothbrushing---3.pkl----
----41---Kitchen---Chopping---1.pkl----
----45---Bathroom---Hair_dryer_in_use---3.pkl----
----17---Kitchen---Grating---1.pkl----
----37---Workshop---Screwing---3.pkl----
----203---Kitchen---Twisting_jar---2.pkl----
----7---Misc---Alarm_clock---2.pkl----
----105---Bathroom---Toothbrushing---3.pkl----
----37---Kitchen---Grating---2.pkl----
----37---Misc---Knocking---2.pkl----
----25---Bathroom---Brushing_hair---2.pkl----
----17---Misc---Alarm_clock---1.pkl----
----53---Workshop---Hammering---1.pkl----
----49---Kitchen---Chopping---3.pkl----
----55---Misc---Knocking---3.pkl----
----201---Bathroom---Toothbrushing---3.pkl----
----201---All---Other---1.pkl----
----203---Workshop---Hammering---3.pkl----
----23---Misc---Coughing---3.pkl----
----53---Other---Other---17.pkl----
----45---Bathroom---Toothbrushing---3.pkl----
----207---Workshop---Sanding---2.pkl----
----4

In [8]:
imu_examples.shape

(54, 100, 9)

In [9]:
# sanity check

In [10]:
for path_to_subsr in path_to_save.iterdir():
    for fpath in path_to_subsr.iterdir():
        with open(fpath, "rb") as f:
            fdata = pkl.load(f)
            print(fdata["IMU"].shape, fdata["audio"].shape)

(60, 100, 9) (60, 96, 64)
(48, 100, 9) (48, 96, 64)
(41, 100, 9) (41, 96, 64)
(26, 100, 9) (26, 96, 64)
(72, 100, 9) (72, 96, 64)
(66, 100, 9) (66, 96, 64)
(16, 100, 9) (16, 96, 64)
(15, 100, 9) (15, 96, 64)
(53, 100, 9) (53, 96, 64)
(19, 100, 9) (19, 96, 64)
(62, 100, 9) (62, 96, 64)
(34, 100, 9) (34, 96, 64)
(64, 100, 9) (64, 96, 64)
(71, 100, 9) (71, 96, 64)
(37, 100, 9) (37, 96, 64)
(69, 100, 9) (69, 96, 64)
(53, 100, 9) (53, 96, 64)
(38, 100, 9) (38, 96, 64)
(71, 100, 9) (71, 96, 64)
(65, 100, 9) (65, 96, 64)
(30, 100, 9) (30, 96, 64)
(84, 100, 9) (84, 96, 64)
(73, 100, 9) (73, 96, 64)
(64, 100, 9) (64, 96, 64)
(64, 100, 9) (64, 96, 64)
(69, 100, 9) (69, 96, 64)
(77, 100, 9) (77, 96, 64)
(34, 100, 9) (34, 96, 64)
(77, 100, 9) (77, 96, 64)
(91, 100, 9) (91, 96, 64)
(65, 100, 9) (65, 96, 64)
(26, 100, 9) (26, 96, 64)
(69, 100, 9) (69, 96, 64)
(70, 100, 9) (70, 96, 64)
(62, 100, 9) (62, 96, 64)
(67, 100, 9) (67, 96, 64)
(26, 100, 9) (26, 96, 64)
(56, 100, 9) (56, 96, 64)
(66, 100, 9)

(60, 100, 9) (60, 96, 64)
(79, 100, 9) (79, 96, 64)
(39, 100, 9) (39, 96, 64)
(67, 100, 9) (67, 96, 64)
(9, 100, 9) (9, 96, 64)
(82, 100, 9) (82, 96, 64)
(41, 100, 9) (41, 96, 64)
(19, 100, 9) (19, 96, 64)
(68, 100, 9) (68, 96, 64)
(23, 100, 9) (23, 96, 64)
(72, 100, 9) (72, 96, 64)
(73, 100, 9) (73, 96, 64)
(38, 100, 9) (38, 96, 64)
(67, 100, 9) (67, 96, 64)
(69, 100, 9) (69, 96, 64)
(70, 100, 9) (70, 96, 64)
(64, 100, 9) (64, 96, 64)
(37, 100, 9) (37, 96, 64)
(41, 100, 9) (41, 96, 64)
(55, 100, 9) (55, 96, 64)
(49, 100, 9) (49, 96, 64)
(26, 100, 9) (26, 96, 64)
(65, 100, 9) (65, 96, 64)
(33, 100, 9) (33, 96, 64)
(1, 100, 9) (1, 96, 64)
(8, 100, 9) (8, 96, 64)
(79, 100, 9) (79, 96, 64)
(75, 100, 9) (75, 96, 64)
(14, 100, 9) (14, 96, 64)
(24, 100, 9) (24, 96, 64)
(48, 100, 9) (48, 96, 64)
(36, 100, 9) (36, 96, 64)
(60, 100, 9) (60, 96, 64)
(75, 100, 9) (75, 96, 64)
(42, 100, 9) (42, 96, 64)
(54, 100, 9) (54, 96, 64)
(43, 100, 9) (43, 96, 64)
(7, 100, 9) (7, 96, 64)
(3, 100, 9) (3, 96, 

(64, 100, 9) (64, 96, 64)
(74, 100, 9) (74, 96, 64)
(73, 100, 9) (73, 96, 64)
(56, 100, 9) (56, 96, 64)
(74, 100, 9) (74, 96, 64)
(32, 100, 9) (32, 96, 64)
(123, 100, 9) (123, 96, 64)
(48, 100, 9) (48, 96, 64)
(72, 100, 9) (72, 96, 64)
(10, 100, 9) (10, 96, 64)
(55, 100, 9) (55, 96, 64)
(52, 100, 9) (52, 96, 64)
(34, 100, 9) (34, 96, 64)
(0,) (0,)
(38, 100, 9) (38, 96, 64)
(67, 100, 9) (67, 96, 64)
(161, 100, 9) (161, 96, 64)
(29, 100, 9) (29, 96, 64)
(26, 100, 9) (26, 96, 64)
(95, 100, 9) (95, 96, 64)
(79, 100, 9) (79, 96, 64)
(179, 100, 9) (179, 96, 64)
(80, 100, 9) (80, 96, 64)
(62, 100, 9) (62, 96, 64)
(87, 100, 9) (87, 96, 64)
(50, 100, 9) (50, 96, 64)
(43, 100, 9) (43, 96, 64)
(66, 100, 9) (66, 96, 64)
(17, 100, 9) (17, 96, 64)
(22, 100, 9) (22, 96, 64)
(64, 100, 9) (64, 96, 64)
(58, 100, 9) (58, 96, 64)
(71, 100, 9) (71, 96, 64)
(14, 100, 9) (14, 96, 64)
(68, 100, 9) (68, 96, 64)
(70, 100, 9) (70, 96, 64)
(67, 100, 9) (67, 96, 64)
(65, 100, 9) (65, 96, 64)
(46, 100, 9) (46, 96, 

(49, 100, 9) (49, 96, 64)
(49, 100, 9) (49, 96, 64)
(75, 100, 9) (75, 96, 64)
(131, 100, 9) (131, 96, 64)
(68, 100, 9) (68, 96, 64)
(64, 100, 9) (64, 96, 64)
(119, 100, 9) (119, 96, 64)
(36, 100, 9) (36, 96, 64)
(62, 100, 9) (62, 96, 64)
(75, 100, 9) (75, 96, 64)
(91, 100, 9) (91, 96, 64)
(60, 100, 9) (60, 96, 64)
(66, 100, 9) (66, 96, 64)
(57, 100, 9) (57, 96, 64)
(99, 100, 9) (99, 96, 64)
(72, 100, 9) (72, 96, 64)
(44, 100, 9) (44, 96, 64)
(76, 100, 9) (76, 96, 64)
(127, 100, 9) (127, 96, 64)
(40, 100, 9) (40, 96, 64)
(49, 100, 9) (49, 96, 64)
(98, 100, 9) (98, 96, 64)
(10, 100, 9) (10, 96, 64)
(53, 100, 9) (53, 96, 64)
(64, 100, 9) (64, 96, 64)
(1, 100, 9) (1, 96, 64)
(6, 100, 9) (6, 96, 64)
(65, 100, 9) (65, 96, 64)
(67, 100, 9) (67, 96, 64)
(93, 100, 9) (93, 96, 64)
(48, 100, 9) (48, 96, 64)
(91, 100, 9) (91, 96, 64)
(60, 100, 9) (60, 96, 64)
(76, 100, 9) (76, 96, 64)
(73, 100, 9) (73, 96, 64)
(65, 100, 9) (65, 96, 64)
(7, 100, 9) (7, 96, 64)
(11, 100, 9) (11, 96, 64)
(91, 100, 9)

(38, 100, 9) (38, 96, 64)
(102, 100, 9) (102, 96, 64)
(86, 100, 9) (86, 96, 64)
(44, 100, 9) (44, 96, 64)
(50, 100, 9) (50, 96, 64)
(60, 100, 9) (60, 96, 64)
(59, 100, 9) (59, 96, 64)
(36, 100, 9) (36, 96, 64)
(72, 100, 9) (72, 96, 64)
(65, 100, 9) (65, 96, 64)
(41, 100, 9) (41, 96, 64)
(109, 100, 9) (109, 96, 64)
(69, 100, 9) (69, 96, 64)
(65, 100, 9) (65, 96, 64)
(12, 100, 9) (12, 96, 64)
(25, 100, 9) (25, 96, 64)
(30, 100, 9) (30, 96, 64)
(75, 100, 9) (75, 96, 64)
(69, 100, 9) (69, 96, 64)
(63, 100, 9) (63, 96, 64)
(60, 100, 9) (60, 96, 64)
(71, 100, 9) (71, 96, 64)
(37, 100, 9) (37, 96, 64)
(67, 100, 9) (67, 96, 64)
(65, 100, 9) (65, 96, 64)
(69, 100, 9) (69, 96, 64)
(65, 100, 9) (65, 96, 64)
(71, 100, 9) (71, 96, 64)
(71, 100, 9) (71, 96, 64)
(70, 100, 9) (70, 96, 64)
(141, 100, 9) (141, 96, 64)
(24, 100, 9) (24, 96, 64)
(17, 100, 9) (17, 96, 64)
(40, 100, 9) (40, 96, 64)
(10, 100, 9) (10, 96, 64)
(85, 100, 9) (85, 96, 64)
(11, 100, 9) (11, 96, 64)
(75, 100, 9) (75, 96, 64)
(72, 1

...

(91, 100, 9) (91, 96, 64)
(67, 100, 9) (67, 96, 64)
(38, 100, 9) (38, 96, 64)
(70, 100, 9) (70, 96, 64)
(80, 100, 9) (80, 96, 64)
(68, 100, 9) (68, 96, 64)
(86, 100, 9) (86, 96, 64)
(41, 100, 9) (41, 96, 64)
(76, 100, 9) (76, 96, 64)
(73, 100, 9) (73, 96, 64)
(0,) (0,)
(75, 100, 9) (75, 96, 64)
(28, 100, 9) (28, 96, 64)
(6, 100, 9) (6, 96, 64)
(36, 100, 9) (36, 96, 64)
(145, 100, 9) (145, 96, 64)
(39, 100, 9) (39, 96, 64)
(50, 100, 9) (50, 96, 64)
(55, 100, 9) (55, 96, 64)
(79, 100, 9) (79, 96, 64)
(66, 100, 9) (66, 96, 64)
(95, 100, 9) (95, 96, 64)
(88, 100, 9) (88, 96, 64)
(62, 100, 9) (62, 96, 64)
(29, 100, 9) (29, 96, 64)
(69, 100, 9) (69, 96, 64)
(49, 100, 9) (49, 96, 64)
(3, 100, 9) (3, 96, 64)
(41, 100, 9) (41, 96, 64)
(25, 100, 9) (25, 96, 64)
(3, 100, 9) (3, 96, 64)
(65, 100, 9) (65, 96, 64)
(75, 100, 9) (75, 96, 64)
(52, 100, 9) (52, 96, 64)
(90, 100, 9) (90, 96, 64)
(9, 100, 9) (9, 96, 64)
(69, 100, 9) (69, 96, 64)
(9, 100, 9) (9, 96, 64)
(45, 100, 9) (45, 96, 64)
(20, 100, 

(40, 100, 9) (40, 96, 64)
(198, 100, 9) (198, 96, 64)
(44, 100, 9) (44, 96, 64)
(41, 100, 9) (41, 96, 64)
(31, 100, 9) (31, 96, 64)
(67, 100, 9) (67, 96, 64)
(72, 100, 9) (72, 96, 64)
(9, 100, 9) (9, 96, 64)
(71, 100, 9) (71, 96, 64)
(70, 100, 9) (70, 96, 64)
(13, 100, 9) (13, 96, 64)
(67, 100, 9) (67, 96, 64)
(76, 100, 9) (76, 96, 64)
(68, 100, 9) (68, 96, 64)
(3, 100, 9) (3, 96, 64)
(69, 100, 9) (69, 96, 64)
(56, 100, 9) (56, 96, 64)
(66, 100, 9) (66, 96, 64)
(53, 100, 9) (53, 96, 64)
(99, 100, 9) (99, 96, 64)
(69, 100, 9) (69, 96, 64)
(70, 100, 9) (70, 96, 64)
(63, 100, 9) (63, 96, 64)
(39, 100, 9) (39, 96, 64)
(80, 100, 9) (80, 96, 64)
(32, 100, 9) (32, 96, 64)
(86, 100, 9) (86, 96, 64)
(4, 100, 9) (4, 96, 64)
(84, 100, 9) (84, 96, 64)
(6, 100, 9) (6, 96, 64)
(15, 100, 9) (15, 96, 64)
(53, 100, 9) (53, 96, 64)
(67, 100, 9) (67, 96, 64)
(66, 100, 9) (66, 96, 64)
(71, 100, 9) (71, 96, 64)
(31, 100, 9) (31, 96, 64)
(43, 100, 9) (43, 96, 64)
(54, 100, 9) (54, 96, 64)
(44, 100, 9) (44, 

(5, 100, 9) (5, 96, 64)
(111, 100, 9) (111, 96, 64)
(37, 100, 9) (37, 96, 64)
(30, 100, 9) (30, 96, 64)
(64, 100, 9) (64, 96, 64)
(3, 100, 9) (3, 96, 64)
(49, 100, 9) (49, 96, 64)
(102, 100, 9) (102, 96, 64)
(68, 100, 9) (68, 96, 64)
(73, 100, 9) (73, 96, 64)
(64, 100, 9) (64, 96, 64)
(59, 100, 9) (59, 96, 64)
(17, 100, 9) (17, 96, 64)
(74, 100, 9) (74, 96, 64)
(64, 100, 9) (64, 96, 64)
(37, 100, 9) (37, 96, 64)
(64, 100, 9) (64, 96, 64)
(50, 100, 9) (50, 96, 64)
(64, 100, 9) (64, 96, 64)
(4, 100, 9) (4, 96, 64)
(97, 100, 9) (97, 96, 64)
(57, 100, 9) (57, 96, 64)
(66, 100, 9) (66, 96, 64)
(69, 100, 9) (69, 96, 64)
(45, 100, 9) (45, 96, 64)
(71, 100, 9) (71, 96, 64)
(64, 100, 9) (64, 96, 64)
(44, 100, 9) (44, 96, 64)
(67, 100, 9) (67, 96, 64)
(19, 100, 9) (19, 96, 64)
(41, 100, 9) (41, 96, 64)
(66, 100, 9) (66, 96, 64)
(48, 100, 9) (48, 96, 64)
(74, 100, 9) (74, 96, 64)
(42, 100, 9) (42, 96, 64)
(0,) (0,)
(37, 100, 9) (37, 96, 64)
(73, 100, 9) (73, 96, 64)
(65, 100, 9) (65, 96, 64)
(13,

(68, 100, 9) (68, 96, 64)
(77, 100, 9) (77, 96, 64)
(78, 100, 9) (78, 96, 64)
(70, 100, 9) (70, 96, 64)
(69, 100, 9) (69, 96, 64)
(178, 100, 9) (178, 96, 64)
(69, 100, 9) (69, 96, 64)
(72, 100, 9) (72, 96, 64)
(47, 100, 9) (47, 96, 64)
(17, 100, 9) (17, 96, 64)
(55, 100, 9) (55, 96, 64)
(267, 100, 9) (267, 96, 64)
(50, 100, 9) (50, 96, 64)
(64, 100, 9) (64, 96, 64)
(71, 100, 9) (71, 96, 64)
(42, 100, 9) (42, 96, 64)
(13, 100, 9) (13, 96, 64)
(69, 100, 9) (69, 96, 64)
(67, 100, 9) (67, 96, 64)
(40, 100, 9) (40, 96, 64)
(6, 100, 9) (6, 96, 64)
(7, 100, 9) (7, 96, 64)
(29, 100, 9) (29, 96, 64)
(0,) (0,)
(61, 100, 9) (61, 96, 64)
(54, 100, 9) (54, 96, 64)
(86, 100, 9) (86, 96, 64)
(52, 100, 9) (52, 96, 64)
(16, 100, 9) (16, 96, 64)
(75, 100, 9) (75, 96, 64)
(66, 100, 9) (66, 96, 64)
(33, 100, 9) (33, 96, 64)
(260, 100, 9) (260, 96, 64)
(79, 100, 9) (79, 96, 64)
(72, 100, 9) (72, 96, 64)
(78, 100, 9) (78, 96, 64)
(39, 100, 9) (39, 96, 64)
(74, 100, 9) (74, 96, 64)
(79, 100, 9) (79, 96, 64)


(69, 100, 9) (69, 96, 64)
(52, 100, 9) (52, 96, 64)
(71, 100, 9) (71, 96, 64)
(84, 100, 9) (84, 96, 64)
(74, 100, 9) (74, 96, 64)
(15, 100, 9) (15, 96, 64)
(43, 100, 9) (43, 96, 64)
(4, 100, 9) (4, 96, 64)
(28, 100, 9) (28, 96, 64)
(67, 100, 9) (67, 96, 64)
(0,) (0,)
(41, 100, 9) (41, 96, 64)
(33, 100, 9) (33, 96, 64)
(75, 100, 9) (75, 96, 64)
(37, 100, 9) (37, 96, 64)
(169, 100, 9) (169, 96, 64)
(51, 100, 9) (51, 96, 64)
(8, 100, 9) (8, 96, 64)
(70, 100, 9) (70, 96, 64)
(77, 100, 9) (77, 96, 64)
(67, 100, 9) (67, 96, 64)
(73, 100, 9) (73, 96, 64)
(74, 100, 9) (74, 96, 64)
(57, 100, 9) (57, 96, 64)
(65, 100, 9) (65, 96, 64)
(34, 100, 9) (34, 96, 64)
(111, 100, 9) (111, 96, 64)
(44, 100, 9) (44, 96, 64)
(94, 100, 9) (94, 96, 64)
(49, 100, 9) (49, 96, 64)
(68, 100, 9) (68, 96, 64)
(50, 100, 9) (50, 96, 64)
(66, 100, 9) (66, 96, 64)
(96, 100, 9) (96, 96, 64)
(38, 100, 9) (38, 96, 64)
(47, 100, 9) (47, 96, 64)
(56, 100, 9) (56, 96, 64)
(62, 100, 9) (62, 96, 64)
(73, 100, 9) (73, 96, 64)
(6